**Objective of the notebook:**

In this notebook, let us explore the given dataset and make some inferences along the way. Also finally we will build a baseline light gbm model to get started. 

**Objective of the competition:**

In this competition, we a’re challenged to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. 

In [1]:
import os
#os.getcwd()
os.chdir("/Users/JidaThada/Desktop/2018 Fall/Machine Learning/Project/google_analytics")

In [2]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_selection import f_classif
#import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

**About the dataset:**

Similar to most other kaggle competitions, we are given two datasets
* train.csv
* test.csv

Each row in the dataset is one visit to the store. We are predicting the natural log of the sum of all transactions per user. 
    
The data fields in the given files are 
* fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
* channelGrouping - The channel via which the user came to the Store.
* date - The date on which the user visited the Store.
* device - The specifications for the device used to access the Store.
* geoNetwork - This section contains information about the geography of the user.
* sessionId - A unique identifier for this visit to the store.
* socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* totals - This section contains aggregate values across the session.
* trafficSource - This section contains information about the Traffic Source from which the session originated.
* visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
* visitNumber - The session number for this user. If this is the first session, then this is set to 1.
* visitStartTime - The timestamp (expressed as POSIX time).

Also it is important to note that some of the fields are in json format. 

Thanks to this [wonderful kernel](https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook) by [Julian](https://www.kaggle.com/julian3833), we can convert all the json fields in the file to a flattened csv format which generally use in other competitions.

In [ ]:
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
train_df = load_df()
test_df = load_df("../input/test.csv")

In [ ]:
train_df.to_csv('train_flatten.csv')
test_df.to_csv('test_flatten.csv')



Gets data from the dataset

In [3]:
train_df = pd.read_csv("train_flatten.csv", dtype={'fullVisitorId': 'str'})
test_df = pd.read_csv("test_flatten.csv", dtype={'fullVisitorId': 'str'})


/Users/JidaThada/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
len(set(test_df['fullVisitorId']))

617242

In [5]:
sample_submission = pd.read_csv("sample_submission.csv")
len(set(sample_submission['fullVisitorId']))


617242

In [6]:
processed_test_df = pd.read_csv("processed_test_df.csv", dtype={'fullVisitorId': 'str'})
len(set(processed_test_df['fullVisitorId']))

620627

In [9]:
train_df.head()

,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Izmir,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,(not set),ttnet.com.tr,not available in demo dataset,Izmir,Western Asia,1.0,1,1.0,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Oceania,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,Australasia,1.0,1,1.0,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Madrid,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Community of Madrid,Southern Europe,1.0,1,1.0,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not 

In [7]:
new_train_df = train_df.copy()

In [8]:
num_cols = ["totals.hits", "totals.pageviews", "visitNumber"]
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)


# change date to get only month
def changeDateToMonth(ts):
    return str(ts)[4:6]
new_train_df['date'] = new_train_df['date'].apply(changeDateToMonth)

# changing visitStartTime from POSIX timestamp to month
def changePOSIXtoHour(ts):
    return datetime.utcfromtimestamp(ts).strftime('%H')
new_train_df['visitStartTime'] = new_train_df['visitStartTime'].apply(changePOSIXtoHour)

# replace 'totals.bounces' : 1->0, nan->1
def switch(ts):
    if ts == 1:
        return 0
    else:
        return 1
new_train_df['totals.bounces'] = new_train_df['totals.bounces'].apply(switch)

def isNewVisit(val):
    if(val != 1):
        return 0
    else:
        return 1
new_train_df['totals.newVisits'] = new_train_df['totals.newVisits'].apply(isNewVisit)
new_train_df.head()

,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,09,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,15,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Izmir,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,(not set),ttnet.com.tr,not available in demo dataset,Izmir,Western Asia,0,1,1,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,09,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,05,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Oceania,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,Australasia,0,1,1,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,09,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,01,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Madrid,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Community of Madrid,Southern Europe,0,1,1,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,09,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,05,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not av

In [9]:
new_train_df["totals.transactionRevenue"].fillna(0, inplace=True)
new_train_df.head()

,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,09,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,15,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Izmir,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,(not set),ttnet.com.tr,not available in demo dataset,Izmir,Western Asia,0,1,1,1.0,0.0,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,09,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,05,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Oceania,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,Australasia,0,1,1,1.0,0.0,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,09,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,01,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Madrid,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Community of Madrid,Southern Europe,0,1,1,1.0,0.0,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,09,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,05,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not av

In [10]:
# drop columns
const_cols = [c for c in new_train_df.columns if new_train_df[c].nunique(dropna=False)==1 ]
#print(const_cols)

cols_to_drop = const_cols + ['sessionId'] + ['visitId']

new_train_df = new_train_df.drop(cols_to_drop + ["trafficSource.campaignCode"], axis=1)

In [11]:
new2_train_df = new_train_df.copy()

In [12]:
new2_train_df.head()

,Unnamed: 0,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,09,1131660440785968503,1,15,Chrome,desktop,False,Windows,Izmir,Asia,Turkey,(not set),ttnet.com.tr,Izmir,Western Asia,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,1,Organic Search,09,377306020877927890,1,05,Firefox,desktop,False,Macintosh,not available in demo dataset,Oceania,Australia,not available in demo dataset,dodo.net.au,not available in demo dataset,Australasia,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,2,Organic Search,09,3895546263509774583,1,01,Chrome,desktop,False,Windows,Madrid,Europe,Spain,(not set),unknown.unknown,Community of Madrid,Southern Europe,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,3,Organic Search,09,4763447161404445595,1,05,UC Browser,desktop,False,Linux,not available in demo dataset,Asia,Indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,Southeast Asia,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google
4,4,Organic Search,09,27294437909732085,2,13,Chrome,mobile,True,Android,not available in demo dataset,Europe,United Kingdom,not available in demo dataset,unknown.unknown,not available in demo dataset,Northern Europe,0,1,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


In [13]:
new2_train_df = new2_train_df.drop('Unnamed: 0', axis=1)
new2_train_df.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,09,1131660440785968503,1,15,Chrome,desktop,False,Windows,Izmir,Asia,Turkey,(not set),ttnet.com.tr,Izmir,Western Asia,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,09,377306020877927890,1,05,Firefox,desktop,False,Macintosh,not available in demo dataset,Oceania,Australia,not available in demo dataset,dodo.net.au,not available in demo dataset,Australasia,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,09,3895546263509774583,1,01,Chrome,desktop,False,Windows,Madrid,Europe,Spain,(not set),unknown.unknown,Community of Madrid,Southern Europe,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,09,4763447161404445595,1,05,UC Browser,desktop,False,Linux,not available in demo dataset,Asia,Indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,Southeast Asia,0,1,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google
4,Organic Search,09,27294437909732085,2,13,Chrome,mobile,True,Android,not available in demo dataset,Europe,United Kingdom,not available in demo dataset,unknown.unknown,not available in demo dataset,Northern Europe,0,1,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


In [11]:
new2_train_df.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'visitNumber',
       'visitStartTime', 'device.browser', 'device.deviceCategory',
       'device.isMobile', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [14]:
num_cols = ["totals.hits", "totals.pageviews", "visitNumber"]
tf_cat_cols = ['date',"visitStartTime",'totals.newVisits','totals.bounces']
real_cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
new_col = ['fullVisitorId', 'totals.transactionRevenue']+num_cols+tf_cat_cols+real_cat_cols
print(new_col)

['fullVisitorId', 'totals.transactionRevenue', 'totals.hits', 'totals.pageviews', 'visitNumber', 'date', 'visitStartTime', 'totals.newVisits', 'totals.bounces', 'channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.page', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']


In [15]:
new2_train_df = new2_train_df[new_col]
new2_train_df.head()

,fullVisitorId,totals.transactionRevenue,totals.hits,totals.pageviews,visitNumber,date,visitStartTime,totals.newVisits,totals.bounces,channelGrouping,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect
0,1131660440785968503,0.0,1,1.0,1,09,15,1,0,Organic Search,Chrome,desktop,Windows,Izmir,Asia,Turkey,(not set),ttnet.com.tr,Izmir,Western Asia,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN
1,377306020877927890,0.0,1,1.0,1,09,05,1,0,Organic Search,Firefox,desktop,Macintosh,not available in demo dataset,Oceania,Australia,not available in demo dataset,dodo.net.au,not available in demo dataset,Australasia,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN
2,3895546263509774583,0.0,1,1.0,1,09,01,1,0,Organic Search,Chrome,desktop,Windows,Madrid,Europe,Spain,(not set),unknown.unknown,Community of Madrid,Southern Europe,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN
3,4763447161404445595,0.0,1,1.0,1,09,05,1,0,Organic Search,UC Browser,desktop,Linux,not available in demo dataset,Asia,Indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,Southeast Asia,NaN,NaN,NaN,NaN,NaN,(not set),google + online,organic,NaN,google,NaN,NaN
4,27294437909732085,0.0,1,1.0,2,09,13,0,0,Organic Search,Chrome,mobile,Android,not available in demo dataset,Europe,United Kingdom,not available in demo dataset,unknown.unknown,not available in demo dataset,Northern Europe,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,True


In [16]:
# correlation test btw num vs target
non_zero_df = new2_train_df[new2_train_df['totals.transactionRevenue']>0]
for col in num_cols:
    print(col, np.corrcoef(non_zero_df[col], non_zero_df['totals.transactionRevenue'])[0,1])

totals.hits 0.1419734837008005
totals.pageviews 0.12864390053779454
visitNumber 0.3088951874376234


In [17]:
new2_train_df['totals.pageviews'].fillna(0, inplace=True)

In [18]:
new_test_df = test_df.copy()
new_test_df.head()

,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2.0,1508151024,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,Asia,Singapore,not available in demo dataset,not available in demo dataset,(not set),myrepublic.com.sg,not available in demo dataset,(not set),Southeast Asia,NaN,4.0,NaN,4.0,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
1,1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1.0,1508175522,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Zaragoza,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),rima-tde.net,not available in demo dataset,Aragon,Southern Europe,NaN,5.0,1.0,5.0,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1.0,1508143220,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,France,not available in demo dataset,not available in demo dataset,not available in demo dataset,sfr.net,not available in demo dataset,not available in demo dataset,Western Europe,NaN,7.0,1.0,7.0,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1.0,1508193530,Safari,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not 

In [19]:
# Transform data on Testset

new_test_df['date'] = new_test_df['date'].apply(changeDateToMonth)
new_test_df['visitStartTime'] = new_test_df['visitStartTime'].apply(changePOSIXtoHour)
new_test_df['totals.bounces'] = new_test_df['totals.bounces'].apply(switch)
new_test_df['totals.newVisits'] = new_test_df['totals.newVisits'].apply(isNewVisit)

new_test_df = new_test_df.drop(cols_to_drop, axis=1)
new2_test_df = new_test_df.copy()
new2_test_df = new2_test_df.drop('Unnamed: 0', axis=1)
new_col.remove('totals.transactionRevenue')
new2_test_df = new2_test_df[new_col]
new2_test_df.head()

,fullVisitorId,totals.hits,totals.pageviews,visitNumber,date,visitStartTime,totals.newVisits,totals.bounces,channelGrouping,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect
0,6167871330617112363,4.0,4.0,2.0,10,10,0,1,Organic Search,Chrome,desktop,Macintosh,(not set),Asia,Singapore,(not set),myrepublic.com.sg,(not set),Southeast Asia,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,True
1,0643697640977915618,5.0,5.0,1.0,10,17,1,1,Organic Search,Chrome,desktop,Windows,Zaragoza,Europe,Spain,(not set),rima-tde.net,Aragon,Southern Europe,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN
2,6059383810968229466,7.0,7.0,1.0,10,08,1,1,Organic Search,Chrome,desktop,Macintosh,not available in demo dataset,Europe,France,not available in demo dataset,sfr.net,not available in demo dataset,Western Europe,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN
3,2376720078563423631,8.0,4.0,1.0,10,22,1,1,Organic Search,Safari,mobile,iOS,Mountain View,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN
4,2314544520795440038,9.0,4.0,1.0,10,05,1,1,Organic Search,Safari,desktop,Macintosh,San Jose,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN


In [20]:
# LabelEncode for cat
cat_cols = tf_cat_cols + real_cat_cols
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(new2_train_df[col].values.astype('str')) + list(new2_test_df[col].values.astype('str')))
    new2_train_df[col] = lbl.transform(list(new2_train_df[col].values.astype('str')))
    new2_test_df[col] = lbl.transform(list(new2_test_df[col].values.astype('str')))
new2_train_df.head()

date
visitStartTime
totals.newVisits
totals.bounces
channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


,fullVisitorId,totals.transactionRevenue,totals.hits,totals.pageviews,visitNumber,date,visitStartTime,totals.newVisits,totals.bounces,channelGrouping,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect
0,1131660440785968503,0.0,1,1.0,1,8,15,1,0,4,35,0,20,378,3,210,0,37454,193,21,62,3,59008,11,3,4,11,5,3196,208,1,1
1,377306020877927890,0.0,1,1.0,1,8,5,1,0,4,43,0,7,955,5,12,122,10098,482,1,62,3,59008,11,3,4,11,5,3196,208,1,1
2,3895546263509774583,0.0,1,1.0,1,8,1,1,0,4,35,0,20,475,4,185,0,38725,99,19,62,3,59008,11,3,4,11,5,3196,208,1,1
3,4763447161404445595,0.0,1,1.0,1,8,5,1,0,4,79,0,6,955,3,94,122,38725,482,16,62,3,59008,11,3,4,1607,5,3196,208,1,1
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,35,1,1,955,4,217,122,38725,482,13,62,3,59008,11,3,4,11,5,3196,208,1,0


In [23]:
# ANOVA test btw cat vs target
non_zero_df = new2_train_df[new2_train_df['totals.transactionRevenue']>0]
cat_cols = tf_cat_cols + real_cat_cols
X = non_zero_df[cat_cols]
f_val, p_val = f_classif(X, non_zero_df['totals.transactionRevenue'])
for i in range(len(cat_cols)):
    print(cat_cols[i], p_val[i])

date 0.9999975586366356
visitStartTime 0.8879582304838451
totals.newVisits 0.07158962744299657
totals.bounces nan
channelGrouping 1.5982249993098262e-99
device.browser 0.9999999978946931
device.deviceCategory 0.9999999070503746
device.operatingSystem 9.190340155750066e-10
geoNetwork.city 3.706839894025699e-27
geoNetwork.continent 0.9999998310935709
geoNetwork.country 0.0014894079136838752
geoNetwork.metro 9.851973035547985e-26
geoNetwork.networkDomain 1.1136050842423247e-06
geoNetwork.region 2.098098324133557e-30
geoNetwork.subContinent 1.7605743635978588e-08
trafficSource.adContent 0.999999999999997
trafficSource.adwordsClickInfo.adNetworkType 0.9999946324034794
trafficSource.adwordsClickInfo.gclId 0.9992398767731581
trafficSource.adwordsClickInfo.page 0.9999946324034793
trafficSource.adwordsClickInfo.slot 0.9999987033803517
trafficSource.campaign 4.379064602451089e-14
trafficSource.keyword 3.887887099792786e-126
trafficSource.medium 2.5661250237825036e-59
trafficSource.referralPath 1

/Users/JidaThada/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [3] are constant.

/Users/JidaThada/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning:

invalid value encountered in true_divide



In [24]:
new3_train_df = new2_train_df.copy()
alpha = 0.1
keep_cat_cols = ['date', 'visitStartTime', 'totals.bounces']
for i in range(len(cat_cols)):
    if p_val[i] < alpha: keep_cat_cols.append(cat_cols[i])
new3_train_df = new3_train_df[['fullVisitorId','totals.transactionRevenue']+num_cols+keep_cat_cols]
new3_train_df.head()

,fullVisitorId,totals.transactionRevenue,totals.hits,totals.pageviews,visitNumber,date,visitStartTime,totals.bounces,totals.newVisits,channelGrouping,device.operatingSystem,geoNetwork.city,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208


In [25]:
num_cols
new_num_cols = []
for i in num_cols:
    new_num_cols += ['num.' + i]
new_num_cols

['num.totals.hits', 'num.totals.pageviews', 'num.visitNumber']

In [26]:
def numOrCat(a):
    if a in num_cols:
        return 'num.' + a
    if a in cat_cols:
        return 'cat.' + a
    else:
        return a

In [27]:
new4_train_df = new3_train_df.copy()

In [28]:
new4_train_df.columns = list(map(numOrCat, new3_train_df.columns))
new4_train_df.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208


In [29]:
processed_train_df = new4_train_df
processed_train_df.to_csv('processed_train_df.csv')

In [30]:
new2_test_df.columns = list(map(numOrCat, new2_test_df.columns))
new2_test_df.head()

,fullVisitorId,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.newVisits,cat.totals.bounces,cat.channelGrouping,cat.device.browser,cat.device.deviceCategory,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.continent,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.adContent,cat.trafficSource.adwordsClickInfo.adNetworkType,cat.trafficSource.adwordsClickInfo.gclId,cat.trafficSource.adwordsClickInfo.page,cat.trafficSource.adwordsClickInfo.slot,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source,cat.trafficSource.adwordsClickInfo.isVideoAd,cat.trafficSource.isTrueDirect
0,6167871330617112363,4.0,4.0,2.0,9,10,0,1,4,35,0,7,1,3,176,0,24691,0,16,62,3,59008,11,3,4,11,5,3196,208,1,0
1,0643697640977915618,5.0,5.0,1.0,9,17,1,1,4,35,0,20,948,4,185,0,30734,23,19,62,3,59008,11,3,4,11,5,3196,208,1,1
2,6059383810968229466,7.0,7.0,1.0,9,8,1,1,4,35,0,7,955,4,69,122,32535,482,22,62,3,59008,11,3,4,11,5,3196,208,1,1
3,2376720078563423631,8.0,4.0,1.0,9,22,1,1,4,72,1,23,540,2,218,100,0,73,12,62,3,59008,11,3,4,11,5,3196,208,1,1
4,2314544520795440038,9.0,4.0,1.0,9,5,1,1,4,72,0,7,751,2,218,100,0,73,12,62,3,59008,11,3,4,11,5,3196,208,1,1


In [31]:
new3_test_df = new2_test_df.copy()
for i in new3_test_df.columns:
    if i not in processed_train_df.columns: 
        print(i)
        new3_test_df = new3_test_df.drop(i, axis=1)
new3_test_df.head()

cat.device.browser
cat.device.deviceCategory
cat.geoNetwork.continent
cat.trafficSource.adContent
cat.trafficSource.adwordsClickInfo.adNetworkType
cat.trafficSource.adwordsClickInfo.gclId
cat.trafficSource.adwordsClickInfo.page
cat.trafficSource.adwordsClickInfo.slot
cat.trafficSource.adwordsClickInfo.isVideoAd
cat.trafficSource.isTrueDirect


,fullVisitorId,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.newVisits,cat.totals.bounces,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,6167871330617112363,4.0,4.0,2.0,9,10,0,1,4,7,1,176,0,24691,0,16,4,11,5,3196,208
1,0643697640977915618,5.0,5.0,1.0,9,17,1,1,4,20,948,185,0,30734,23,19,4,11,5,3196,208
2,6059383810968229466,7.0,7.0,1.0,9,8,1,1,4,7,955,69,122,32535,482,22,4,11,5,3196,208
3,2376720078563423631,8.0,4.0,1.0,9,22,1,1,4,23,540,218,100,0,73,12,4,11,5,3196,208
4,2314544520795440038,9.0,4.0,1.0,9,5,1,1,4,7,751,218,100,0,73,12,4,11,5,3196,208


In [32]:
len(set(new3_test_df['fullVisitorId']))

617242

In [33]:
processed_test_df = new3_test_df
processed_test_df['num.totals.pageviews'].fillna(0, inplace=True)
processed_test_df.to_csv('processed_test_df.csv')

In [34]:
processed_test_df.isna().any()

fullVisitorId                     False
num.totals.hits                   False
num.totals.pageviews              False
num.visitNumber                   False
cat.date                          False
cat.visitStartTime                False
cat.totals.newVisits              False
cat.totals.bounces                False
cat.channelGrouping               False
cat.device.operatingSystem        False
cat.geoNetwork.city               False
cat.geoNetwork.country            False
cat.geoNetwork.metro              False
cat.geoNetwork.networkDomain      False
cat.geoNetwork.region             False
cat.geoNetwork.subContinent       False
cat.trafficSource.campaign        False
cat.trafficSource.keyword         False
cat.trafficSource.medium          False
cat.trafficSource.referralPath    False
cat.trafficSource.source          False
dtype: bool

In [ ]:
# one-hot encoder

for col in keep_cat_cols:
    temp_var = pd.get_dummies(new3_train_df[col], prefix=col).iloc[:,:-1]
    new3_train_df = new3_train_df.drop(col, axis=1)
    new3_train_df.append(temp_var, sort=False)
new3_train_df.head()

# Run Linear Regression

In [38]:
from sklearn.metrics import mean_squared_error


# Data
# processed_train_df
# processed_test_df

In [43]:
processed_train_df.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208


In [39]:
from sklearn.linear_model import LinearRegression


In [40]:
index_dev = train_df.index[train_df['date'] <= 20170531]
len(index_dev)
index_val = train_df.index[train_df['date'] > 20170531]
len(index_val)

137946

In [41]:
X_train = processed_train_df.iloc[:,2:]
X_train_dev = X_train.iloc[index_dev,:]
X_train_val = X_train.iloc[index_val,:]
Y_train = processed_train_df['totals.transactionRevenue']
Y_train_dev = Y_train.iloc[index_dev]
Y_train_val = Y_train.iloc[index_val]

X_test = processed_test_df.iloc[:,1:]

In [42]:
log_Y_train = np.log1p(Y_train)
log_Y_train_dev = log_Y_train.iloc[index_dev]
log_Y_train_val = log_Y_train.iloc[index_val]

In [43]:
max(log_Y_train)

23.86437469605166

In [44]:
# normalize 
from sklearn import preprocessing

#X_train_scaled = scale(X_train)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [45]:
X_train_scaled = pd.DataFrame(X_train_scaled)
X_train_scaled.columns = X_train.columns
X_train_scaled_dev = X_train_scaled.iloc[index_dev,:]
X_train_scaled_val = X_train_scaled.iloc[index_val,:]

In [47]:
# Linear regression

X_tr = X_train_dev
Y_tr = log_Y_train_dev
X_tr_val = X_train_val
Y_tr_val = log_Y_train_val
X_te = X_test

linear_reg = LinearRegression().fit(X_tr, Y_tr)
linear_reg_Y_train_dev_pred = linear_reg.predict(X_tr)
#print('score', reg.score(X_tr, Y_tr))
print('train_mse', mean_squared_error(Y_tr, linear_reg_Y_train_dev_pred))
print('train_rmse', np.sqrt(mean_squared_error(Y_tr, linear_reg_Y_train_dev_pred)))

linear_reg_Y_train_val_pred = linear_reg.predict(X_tr_val)
print('val_mse', mean_squared_error(Y_tr_val, linear_reg_Y_train_val_pred))
print('val_rmse', np.sqrt(mean_squared_error(Y_tr_val, linear_reg_Y_train_val_pred)))


# Test set
linear_reg_Y_test_val_pred = linear_reg.predict(X_te)
te_id = processed_test_df.iloc[:,0]


train_mse 3.2098131497682547
train_rmse 1.7915951411432927
val_mse 3.6827163716483153
val_rmse 1.9190404820243672


In [48]:
te_id = processed_test_df.iloc[:,0]
processed_test_df.head()

,fullVisitorId,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.newVisits,cat.totals.bounces,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,6167871330617112363,4.0,4.0,2.0,9,10,0,1,4,7,1,176,0,24691,0,16,4,11,5,3196,208
1,0643697640977915618,5.0,5.0,1.0,9,17,1,1,4,20,948,185,0,30734,23,19,4,11,5,3196,208
2,6059383810968229466,7.0,7.0,1.0,9,8,1,1,4,7,955,69,122,32535,482,22,4,11,5,3196,208
3,2376720078563423631,8.0,4.0,1.0,9,22,1,1,4,23,540,218,100,0,73,12,4,11,5,3196,208
4,2314544520795440038,9.0,4.0,1.0,9,5,1,1,4,7,751,218,100,0,73,12,4,11,5,3196,208


In [52]:
# Test set
X_te = X_test

te_id = processed_test_df.iloc[:,0]
linear_reg_Y_test_val_pred = linear_reg.predict(X_te)
trans_rev = np.exp(linear_reg_Y_test_val_pred) - 1

# PredictedLogRevenue
d = {'fullVisitorId':te_id, 'PredictedLogRevenue':trans_rev}
submit = pd.DataFrame(data=d)
col = ['fullVisitorId', 'PredictedLogRevenue']
submit = submit[col]
submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedLogRevenue"].sum().reset_index())
submit['PredictedLogRevenue'] = np.log1p(submit['PredictedLogRevenue'])
print(submit.shape)
submit.head()
submit.to_csv('submit1.csv', index=False)


/Users/JidaThada/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:14: RuntimeWarning:

invalid value encountered in log1p



(617242, 2)


In [53]:
submit.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.503747
1,0000049363351866189,0.253831
2,0000053049821714864,-0.229559
3,0000059488412965267,0.565368
4,0000085840370633780,-0.344739


In [144]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_sub = sample_submission.copy()

In [131]:
#final = pd.merge(sample_submission, submit, how='inner')
submit_test = submit.copy()
final = pd.merge(sample_submission, submit_test)
final.head()

#submit_test = submit_test[~submit_test['fullVisitorId'].isin(sample_submission['fullVisitorId'])]
#submit_test.shape

,fullVisitorId,PredictedLogRevenue


In [125]:
submit.head()

,fullVisitorId,PredictedLogRevenue
0,259678714014,0.503747
1,3038793126460,0.074133
2,9663019041506,-0.149044
3,10438463470860,0.690744
4,10992980461157,-0.794270


In [154]:
max(test_df['date'])

20180430

In [149]:
def strip(a):
    return a.lstrip('0')
sample_sub = sample_sub['fullVisitorId'].apply(strip)
sample_sub.head()

0      259678714014
1    49363351866189
2    53049821714864
3    59488412965267
4    85840370633780
Name: fullVisitorId, dtype: object

In [152]:
submit_test = submit_test[submit_test['fullVisitorId'].isin(sample_sub)]
submit_test.shape

(174788, 2)

In [96]:
# Linear regression

X_tr = X_train_scaled_dev
Y_tr = log_Y_train_dev
X_tr_val = X_train_scaled_val
Y_tr_val = log_Y_train_val


linear_reg = LinearRegression().fit(X_tr, Y_tr)
linear_reg_Y_train_dev_pred = reg.predict(X_tr)
#print('score', reg.score(X_tr, Y_tr))
print('train_mse', mean_squared_error(Y_tr, linear_reg_Y_train_dev_pred))
print('train_rmse', np.sqrt(mean_squared_error(Y_tr, linear_reg_Y_train_dev_pred)))

linear_reg_Y_train_val_pred = reg.predict(X_tr_val)
print('val_mse', mean_squared_error(Y_tr_val, linear_reg_Y_train_val_pred))
print('val_rmse', np.sqrt(mean_squared_error(Y_tr_val, linear_reg_Y_train_val_pred)))

train_mse 4.095654033808169
train_rmse 2.023772228737258
val_mse 4.767061377532054
val_rmse 2.1833601117387973


In [97]:
# Polynomial regression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X_tr = X_train_dev
Y_tr = log_Y_train_dev
X_tr_val = X_train_scaled_val
Y_tr_val = log_Y_train_val

model = Pipeline([('poly',PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
poly_reg = model.fit(X_tr, Y_tr)
poly_reg_Y_train_dev_pred = poly_reg.predict(X_tr)
print('train_mse', mean_squared_error(Y_tr, poly_reg_Y_train_dev_pred))
print('train_rmse', np.sqrt(mean_squared_error(Y_tr, poly_reg_Y_train_dev_pred)))

poly_reg_Y_train_val_pred = poly_reg.predict(X_tr_val)
print('val_mse', mean_squared_error(Y_tr_val, poly_reg_Y_train_val_pred))
print('val_rmse', np.sqrt(mean_squared_error(Y_tr_val, poly_reg_Y_train_val_pred)))



train_mse 2.9109172718533065
train_rmse 1.7061410468813258
val_mse 5.435150342517552
val_rmse 2.331340889384809


Group Transaction per Visitor

In [ ]:
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
gdf = train_df.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()
gdf.head
plt.figure(figsize=(8,6))
plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["totals.transactionRevenue"].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('TransactionRevenue', fontsize=12)
plt.show()

In [ ]:
const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ]
print(const_cols)

Dropping useless columns

In [ ]:
cols_to_drop = const_cols + ['sessionId']

train_df = train_df.drop(cols_to_drop + ["trafficSource.campaignCode"], axis=1)
test_df = test_df.drop(cols_to_drop, axis=1)


In [ ]:
train_df.shape

In [ ]:
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)


In [ ]:
import datetime
# Split the train dataset into development and valid based on time 
threshold_date = int(datetime.date(2017,5,31).strftime('%Y%m%d'))
dev_df = train_df[train_df['date']<=threshold_date].dropna()
val_df = train_df[train_df['date']>threshold_date].dropna()
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)

dev_X = dev_df[cat_cols + num_cols] 
val_X = val_df[cat_cols + num_cols] 
test_X = test_df[cat_cols + num_cols] 

In [ ]:
from sklearn.manifold import Isomap
embedding = Isomap(n_components=5)
X_transformed = embedding.fit_transform(dev_X[:100])
X_transformed.shape